In [ ]:
!pip install PyQt6 opencv-python mediapipe numpy playsound

In [ ]:
!pip install  opencv-python   

In [ ]:
!pip install mediapipe

In [ ]:
!pip install numpy 

In [ ]:
!pip install  playsound

In [ ]:
!pip install PyQt6

In [ ]:
"""
Smart ATM Guardian - Advanced Security Guard Monitoring System
================================================================
A real-time computer vision application that monitors security personnel
for drowsiness, inattentiveness, and absence using MediaPipe and OpenCV.

Requirements:
pip install PyQt6 opencv-python mediapipe numpy playsound

Usage:
python smart_atm_guardian.py
"""

import sys
import cv2
import mediapipe as mp
import numpy as np
from datetime import datetime
from collections import deque
import time
from PyQt6.QtWidgets import (QApplication, QMainWindow, QWidget, QVBoxLayout, 
                              QHBoxLayout, QPushButton, QLabel, QTextEdit, QFrame)
from PyQt6.QtCore import QThread, pyqtSignal, Qt, QTimer
from PyQt6.QtGui import QImage, QPixmap, QFont
from typing import Optional, Tuple

# Try to import playsound, but make it optional for testing
try:
    from playsound import playsound
    SOUND_AVAILABLE = True
except ImportError:
    print("Warning: playsound not available. Audio alerts disabled.")
    SOUND_AVAILABLE = False


# ==================== CONFIGURATION ====================
class Config:
    """Configuration parameters for the Smart ATM Guardian system"""
    
    # Video source (0 for webcam, or RTSP URL like "rtsp://camera_ip:554/stream")
    VIDEO_SOURCE = 0
    
    # Detection thresholds (in seconds)
    DROWSINESS_HEAD_DOWN_THRESHOLD = 10.0
    DROWSINESS_EYES_CLOSED_THRESHOLD = 3.0
    INATTENTIVENESS_THRESHOLD = 60.0
    ABSENCE_THRESHOLD = 120.0
    
    # Alert escalation timing (in seconds)
    TIER1_TO_TIER2_DELAY = 15.0
    TIER2_TO_TIER3_DELAY = 30.0
    
    # Eye Aspect Ratio threshold for detecting closed eyes
    EAR_THRESHOLD = 0.2
    
    # Head pose angle thresholds (in degrees)
    HEAD_DOWN_ANGLE = 25.0  # Pitch angle indicating head down
    HEAD_TURNED_ANGLE = 45.0  # Yaw angle indicating head turned away
    
    # ROI for guard presence detection (as fraction of frame: x, y, w, h)
    GUARD_ROI = (0.3, 0.2, 0.4, 0.6)  # Center region of frame
    
    # Sound files (placeholders - user should provide actual files)
    SOUND_WARNING = "warning.wav"  # Gentle beep or cough
    SOUND_ALARM = "wake_up_alarm.wav"  # Loud wake-up sound
    

# ==================== DETECTION ENGINE ====================
class GuardDetector:
    """
    Core detection engine using MediaPipe for pose and face analysis.
    Implements multi-modal detection: drowsiness, inattentiveness, absence.
    """
    
    def __init__(self):
        # Initialize MediaPipe solutions
        self.mp_pose = mp.solutions.pose
        self.mp_face_mesh = mp.solutions.face_mesh
        self.mp_drawing = mp.solutions.drawing_utils
        
        # Create pose and face mesh detectors
        self.pose = self.mp_pose.Pose(
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5
        )
        self.face_mesh = self.mp_face_mesh.FaceMesh(
            max_num_faces=1,
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5
        )
        
        # State tracking
        self.head_down_start_time = None
        self.eyes_closed_start_time = None
        self.head_turned_start_time = None
        self.absence_start_time = None
        
        # Eye landmarks for EAR calculation (left and right eye)
        self.LEFT_EYE_INDICES = [33, 160, 158, 133, 153, 144]
        self.RIGHT_EYE_INDICES = [362, 385, 387, 263, 373, 380]
        
    def calculate_ear(self, eye_landmarks) -> float:
        """
        Calculate Eye Aspect Ratio (EAR) to detect eye closure.
        
        EAR = (||p2-p6|| + ||p3-p5||) / (2 * ||p1-p4||)
        where p1-p6 are the eye landmark points.
        
        Lower EAR values indicate closed eyes.
        """
        # Vertical distances
        v1 = np.linalg.norm(eye_landmarks[1] - eye_landmarks[5])
        v2 = np.linalg.norm(eye_landmarks[2] - eye_landmarks[4])
        
        # Horizontal distance
        h = np.linalg.norm(eye_landmarks[0] - eye_landmarks[3])
        
        # EAR calculation
        ear = (v1 + v2) / (2.0 * h + 1e-6)  # Add epsilon to avoid division by zero
        return ear
    
    def get_head_pose_angles(self, face_landmarks, image_shape) -> Tuple[float, float, float]:
        """
        Calculate head pose angles (pitch, yaw, roll) from face landmarks.
        
        Pitch: Head up/down (positive = looking up, negative = looking down)
        Yaw: Head left/right (positive = turned right, negative = turned left)
        Roll: Head tilt
        """
        h, w = image_shape[:2]
        
        # Key facial landmarks for pose estimation
        # Nose tip, chin, left eye corner, right eye corner, left mouth corner, right mouth corner
        key_indices = [1, 152, 33, 263, 61, 291]
        
        if not face_landmarks:
            return 0.0, 0.0, 0.0
        
        # Extract 2D landmarks
        landmarks_2d = []
        for idx in key_indices:
            lm = face_landmarks.landmark[idx]
            landmarks_2d.append([lm.x * w, lm.y * h])
        landmarks_2d = np.array(landmarks_2d, dtype=np.float64)
        
        # 3D model points (approximate facial structure)
        model_points = np.array([
            (0.0, 0.0, 0.0),           # Nose tip
            (0.0, -330.0, -65.0),      # Chin
            (-225.0, 170.0, -135.0),   # Left eye corner
            (225.0, 170.0, -135.0),    # Right eye corner
            (-150.0, -150.0, -125.0),  # Left mouth corner
            (150.0, -150.0, -125.0)    # Right mouth corner
        ])
        
        # Camera internals (approximate)
        focal_length = w
        center = (w / 2, h / 2)
        camera_matrix = np.array([
            [focal_length, 0, center[0]],
            [0, focal_length, center[1]],
            [0, 0, 1]
        ], dtype=np.float64)
        
        dist_coeffs = np.zeros((4, 1))  # Assuming no lens distortion
        
        # Solve PnP to get rotation vector
        success, rotation_vector, translation_vector = cv2.solvePnP(
            model_points, landmarks_2d, camera_matrix, dist_coeffs
        )
        
        if not success:
            return 0.0, 0.0, 0.0
        
        # Convert rotation vector to rotation matrix
        rotation_matrix, _ = cv2.Rodrigues(rotation_vector)
        
        # Calculate Euler angles
        sy = np.sqrt(rotation_matrix[0, 0] ** 2 + rotation_matrix[1, 0] ** 2)
        singular = sy < 1e-6
        
        if not singular:
            pitch = np.arctan2(rotation_matrix[2, 1], rotation_matrix[2, 2])
            yaw = np.arctan2(-rotation_matrix[2, 0], sy)
            roll = np.arctan2(rotation_matrix[1, 0], rotation_matrix[0, 0])
        else:
            pitch = np.arctan2(-rotation_matrix[1, 2], rotation_matrix[1, 1])
            yaw = np.arctan2(-rotation_matrix[2, 0], sy)
            roll = 0
        
        # Convert to degrees
        pitch = np.degrees(pitch)
        yaw = np.degrees(yaw)
        roll = np.degrees(roll)
        
        return pitch, yaw, roll
    
    def detect_person_in_roi(self, pose_landmarks, image_shape) -> bool:
        """
        Check if a person (guard) is present in the designated ROI.
        """
        if not pose_landmarks:
            return False
        
        h, w = image_shape[:2]
        roi_x, roi_y, roi_w, roi_h = Config.GUARD_ROI
        
        # Check if key body landmarks (shoulders, nose) are in ROI
        key_landmarks = [
            pose_landmarks.landmark[self.mp_pose.PoseLandmark.NOSE],
            pose_landmarks.landmark[self.mp_pose.PoseLandmark.LEFT_SHOULDER],
            pose_landmarks.landmark[self.mp_pose.PoseLandmark.RIGHT_SHOULDER]
        ]
        
        in_roi_count = 0
        for lm in key_landmarks:
            x, y = lm.x, lm.y
            if (roi_x <= x <= roi_x + roi_w) and (roi_y <= y <= roi_y + roi_h):
                in_roi_count += 1
        
        # At least 2 out of 3 key landmarks should be in ROI
        return in_roi_count >= 2
    
    def process_frame(self, frame: np.ndarray) -> dict:
        """
        Process a single frame and detect guard states.
        
        Returns:
            dict with detection results including state, timings, and annotated frame
        """
        current_time = time.time()
        h, w = frame.shape[:2]
        
        # Convert BGR to RGB for MediaPipe
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Process with pose and face mesh
        pose_results = self.pose.process(rgb_frame)
        face_results = self.face_mesh.process(rgb_frame)
        
        # Initialize detection results
        results = {
            'person_detected': False,
            'eyes_closed': False,
            'head_down': False,
            'head_turned': False,
            'drowsiness_duration': 0.0,
            'inattentiveness_duration': 0.0,
            'absence_duration': 0.0,
            'state': 'UNKNOWN',
            'pitch': 0.0,
            'yaw': 0.0,
            'ear': 0.0
        }
        
        # Draw ROI on frame
        roi_x, roi_y, roi_w, roi_h = Config.GUARD_ROI
        cv2.rectangle(frame, 
                     (int(roi_x * w), int(roi_y * h)),
                     (int((roi_x + roi_w) * w), int((roi_y + roi_h) * h)),
                     (0, 255, 0), 2)
        cv2.putText(frame, "Guard Post", (int(roi_x * w), int(roi_y * h) - 10),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
        
        # Check for person presence
        if pose_results.pose_landmarks:
            results['person_detected'] = self.detect_person_in_roi(
                pose_results.pose_landmarks, frame.shape
            )
            
            # Draw pose landmarks
            self.mp_drawing.draw_landmarks(
                frame, pose_results.pose_landmarks, self.mp_pose.POSE_CONNECTIONS
            )
        
        # Absence detection
        if not results['person_detected']:
            if self.absence_start_time is None:
                self.absence_start_time = current_time
            absence_duration = current_time - self.absence_start_time
            results['absence_duration'] = absence_duration
            
            if absence_duration > Config.ABSENCE_THRESHOLD:
                results['state'] = 'ABSENT'
            else:
                results['state'] = 'PERSON_NOT_IN_POST'
            
            # Reset other timers
            self.head_down_start_time = None
            self.eyes_closed_start_time = None
            self.head_turned_start_time = None
            
        else:
            # Person is present, reset absence timer
            self.absence_start_time = None
            
            # Analyze face for drowsiness and attention
            if face_results.multi_face_landmarks:
                face_landmarks = face_results.multi_face_landmarks[0]
                
                # Draw face mesh
                self.mp_drawing.draw_landmarks(
                    frame, face_landmarks, self.mp_face_mesh.FACEMESH_CONTOURS,
                    landmark_drawing_spec=None,
                    connection_drawing_spec=self.mp_drawing.DrawingSpec(
                        color=(0, 255, 0), thickness=1
                    )
                )
                
                # Calculate EAR for both eyes
                left_eye_points = np.array([
                    [face_landmarks.landmark[i].x * w, face_landmarks.landmark[i].y * h]
                    for i in self.LEFT_EYE_INDICES
                ])
                right_eye_points = np.array([
                    [face_landmarks.landmark[i].x * w, face_landmarks.landmark[i].y * h]
                    for i in self.RIGHT_EYE_INDICES
                ])
                
                left_ear = self.calculate_ear(left_eye_points)
                right_ear = self.calculate_ear(right_eye_points)
                avg_ear = (left_ear + right_ear) / 2.0
                results['ear'] = avg_ear
                
                # Check for closed eyes
                if avg_ear < Config.EAR_THRESHOLD:
                    results['eyes_closed'] = True
                    if self.eyes_closed_start_time is None:
                        self.eyes_closed_start_time = current_time
                else:
                    self.eyes_closed_start_time = None
                
                # Get head pose
                pitch, yaw, roll = self.get_head_pose_angles(face_landmarks, frame.shape)
                results['pitch'] = pitch
                results['yaw'] = yaw
                
                # Check for head down (drowsiness indicator)
                if pitch < -Config.HEAD_DOWN_ANGLE:  # Negative pitch = looking down
                    results['head_down'] = True
                    if self.head_down_start_time is None:
                        self.head_down_start_time = current_time
                else:
                    self.head_down_start_time = None
                
                # Check for head turned away (inattentiveness)
                if abs(yaw) > Config.HEAD_TURNED_ANGLE:
                    results['head_turned'] = True
                    if self.head_turned_start_time is None:
                        self.head_turned_start_time = current_time
                else:
                    self.head_turned_start_time = None
                
                # Calculate durations
                if self.eyes_closed_start_time:
                    eyes_closed_duration = current_time - self.eyes_closed_start_time
                else:
                    eyes_closed_duration = 0.0
                
                if self.head_down_start_time:
                    head_down_duration = current_time - self.head_down_start_time
                else:
                    head_down_duration = 0.0
                
                if self.head_turned_start_time:
                    inattentive_duration = current_time - self.head_turned_start_time
                    results['inattentiveness_duration'] = inattentive_duration
                
                # Determine state based on combined indicators
                # Drowsiness = (head down > 10s) OR (eyes closed > 3s)
                if (head_down_duration > Config.DROWSINESS_HEAD_DOWN_THRESHOLD or
                    eyes_closed_duration > Config.DROWSINESS_EYES_CLOSED_THRESHOLD):
                    results['state'] = 'DROWSY'
                    results['drowsiness_duration'] = max(head_down_duration, eyes_closed_duration)
                elif inattentive_duration > Config.INATTENTIVENESS_THRESHOLD:
                    results['state'] = 'INATTENTIVE'
                else:
                    results['state'] = 'ATTENTIVE'
                
                # Display metrics on frame
                cv2.putText(frame, f"EAR: {avg_ear:.2f}", (10, 30),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
                cv2.putText(frame, f"Pitch: {pitch:.1f} Yaw: {yaw:.1f}", (10, 60),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
            else:
                # Face not detected but person present
                results['state'] = 'FACE_NOT_DETECTED'
        
        results['annotated_frame'] = frame
        return results


# ==================== ALERT MANAGER ====================
class AlertManager:
    """
    Manages the tiered alert system with escalation logic.
    """
    
    def __init__(self):
        self.current_alert_tier = 0
        self.tier1_start_time = None
        self.tier2_start_time = None
        self.last_sound_time = 0
        self.sound_cooldown = 2.0  # Minimum seconds between sounds
        
    def play_sound(self, sound_file: str):
        """Play sound file with cooldown to avoid spam"""
        current_time = time.time()
        if current_time - self.last_sound_time >= self.sound_cooldown:
            if SOUND_AVAILABLE:
                try:
                    playsound(sound_file, block=False)
                except Exception as e:
                    print(f"Error playing sound: {e}")
            else:
                print(f"[SOUND] Would play: {sound_file}")
            self.last_sound_time = current_time
    
    def update(self, state: str, duration: float) -> dict:
        """
        Update alert state based on detected condition.
        
        Returns dict with alert_tier, message, and color
        """
        current_time = time.time()
        
        # Reset if state is normal
        if state in ['ATTENTIVE', 'PERSON_NOT_IN_POST']:
            self.current_alert_tier = 0
            self.tier1_start_time = None
            self.tier2_start_time = None
            return {
                'tier': 0,
                'message': f'Status: {state}',
                'color': 'green'
            }
        
        # Handle problematic states
        if state in ['DROWSY', 'INATTENTIVE']:
            # Tier 1: Initial warning
            if self.current_alert_tier == 0:
                self.current_alert_tier = 1
                self.tier1_start_time = current_time
                self.play_sound(Config.SOUND_WARNING)
                return {
                    'tier': 1,
                    'message': f'WARNING: {state} detected!',
                    'color': 'yellow'
                }
            
            # Tier 2: Escalation after delay
            elif self.current_alert_tier == 1:
                tier1_duration = current_time - self.tier1_start_time
                if tier1_duration > Config.TIER1_TO_TIER2_DELAY:
                    self.current_alert_tier = 2
                    self.tier2_start_time = current_time
                    self.play_sound(Config.SOUND_ALARM)
                    return {
                        'tier': 2,
                        'message': f'ALARM: {state} persisting! WAKE UP!',
                        'color': 'red'
                    }
                return {
                    'tier': 1,
                    'message': f'WARNING: {state} ({tier1_duration:.0f}s)',
                    'color': 'yellow'
                }
            
            # Tier 3: Critical escalation
            elif self.current_alert_tier == 2:
                tier2_duration = current_time - self.tier2_start_time
                if tier2_duration > Config.TIER2_TO_TIER3_DELAY:
                    self.current_alert_tier = 3
                    return {
                        'tier': 3,
                        'message': f'CRITICAL: {state} - Supervisor notified!',
                        'color': 'red'
                    }
                return {
                    'tier': 2,
                    'message': f'ALARM: {state} ({tier2_duration:.0f}s)',
                    'color': 'red'
                }
        
        # Handle absence
        if state == 'ABSENT':
            self.current_alert_tier = 3
            return {
                'tier': 3,
                'message': 'CRITICAL: Guard absent from post!',
                'color': 'red'
            }
        
        return {
            'tier': 0,
            'message': f'Status: {state}',
            'color': 'gray'
        }


# ==================== VIDEO PROCESSING THREAD ====================
class VideoProcessorThread(QThread):
    """
    Worker thread for video capture and processing.
    Runs detection engine and emits results to GUI.
    """
    
    frame_processed = pyqtSignal(object)  # Emits detection results dict
    error_occurred = pyqtSignal(str)
    
    def __init__(self, video_source):
        super().__init__()
        self.video_source = video_source
        self.running = False
        self.detector = GuardDetector()
        self.alert_manager = AlertManager()
        
    def run(self):
        """Main processing loop"""
        self.running = True
        cap = cv2.VideoCapture(self.video_source)
        
        if not cap.isOpened():
            self.error_occurred.emit(f"Failed to open video source: {self.video_source}")
            return
        
        while self.running:
            ret, frame = cap.read()
            if not ret:
                self.error_occurred.emit("Failed to read frame from video source")
                break
            
            # Process frame with detector
            results = self.detector.process_frame(frame)
            
            # Update alert system
            alert_info = self.alert_manager.update(
                results['state'],
                results.get('drowsiness_duration', 0.0) or 
                results.get('inattentiveness_duration', 0.0) or
                results.get('absence_duration', 0.0)
            )
            
            # Combine results
            results['alert'] = alert_info
            
            # Emit to GUI
            self.frame_processed.emit(results)
        
        cap.release()
    
    def stop(self):
        """Stop the processing thread"""
        self.running = False


# ==================== MAIN GUI APPLICATION ====================
class SmartATMGuardian(QMainWindow):
    """
    Main application window for the Smart ATM Guardian system.
    """
    
    def __init__(self):
        super().__init__()
        self.video_thread = None
        self.init_ui()
        
    def init_ui(self):
        """Initialize the user interface"""
        self.setWindowTitle("Smart ATM Guardian - Security Monitoring System")
        self.setGeometry(100, 100, 1200, 800)
        
        # Central widget
        central_widget = QWidget()
        self.setCentralWidget(central_widget)
        main_layout = QVBoxLayout(central_widget)
        
        # Title
        title_label = QLabel("🛡️ Smart ATM Guardian")
        title_font = QFont("Arial", 20, QFont.Weight.Bold)
        title_label.setFont(title_font)
        title_label.setAlignment(Qt.AlignmentFlag.AlignCenter)
        main_layout.addWidget(title_label)
        
        # Video and info layout
        content_layout = QHBoxLayout()
        
        # Left: Video display
        video_layout = QVBoxLayout()
        self.video_label = QLabel()
        self.video_label.setMinimumSize(640, 480)
        self.video_label.setStyleSheet("border: 2px solid #333; background-color: black;")
        self.video_label.setAlignment(Qt.AlignmentFlag.AlignCenter)
        video_layout.addWidget(self.video_label)
        
        # Control buttons
        button_layout = QHBoxLayout()
        self.start_button = QPushButton("▶ Start Monitoring")
        self.start_button.setStyleSheet("""
            QPushButton {
                background-color: #4CAF50;
                color: white;
                font-size: 16px;
                padding: 10px;
                border-radius: 5px;
            }
            QPushButton:hover {
                background-color: #45a049;
            }
        """)
        self.start_button.clicked.connect(self.start_monitoring)
        
        self.stop_button = QPushButton("⏹ Stop Monitoring")
        self.stop_button.setStyleSheet("""
            QPushButton {
                background-color: #f44336;
                color: white;
                font-size: 16px;
                padding: 10px;
                border-radius: 5px;
            }
            QPushButton:hover {
                background-color: #da190b;
            }
            QPushButton:disabled {
                background-color: #cccccc;
            }
        """)
        self.stop_button.clicked.connect(self.stop_monitoring)
        self.stop_button.setEnabled(False)
        
        button_layout.addWidget(self.start_button)
        button_layout.addWidget(self.stop_button)
        video_layout.addLayout(button_layout)
        
        content_layout.addLayout(video_layout, 2)
        
        # Right: Status and event log
        info_layout = QVBoxLayout()
        
        # Status panel
        status_frame = QFrame()
        status_frame.setStyleSheet("border: 2px solid #333; border-radius: 5px; padding: 10px;")
        status_layout = QVBoxLayout(status_frame)
        
        status_title = QLabel("System Status")
        status_title.setFont(QFont("Arial", 14, QFont.Weight.Bold))
        status_layout.addWidget(status_title)
        
        self.status_label = QLabel("Status: Not Monitoring")
        self.status_label.setFont(QFont("Arial", 12))
        self.status_label.setStyleSheet("color: gray; padding: 10px;")
        self.status_label.setWordWrap(True)
        status_layout.addWidget(self.status_label)
        
        self.metrics_label = QLabel("Metrics: --")
        self.metrics_label.setFont(QFont("Arial", 10))
        self.metrics_label.setWordWrap(True)
        status_layout.addWidget(self.metrics_label)
        
        info_layout.addWidget(status_frame)
        
        # Event log
        log_title = QLabel("Event Log")
        log_title.setFont(QFont("Arial", 14, QFont.Weight.Bold))
        info_layout.addWidget(log_title)
        
        self.event_log = QTextEdit()
        self.event_log.setReadOnly(True)
        self.event_log.setStyleSheet("border: 2px solid #333; border-radius: 5px;")
        info_layout.addWidget(self.event_log)
        
        content_layout.addLayout(info_layout, 1)
        
        main_layout.addLayout(content_layout)
        
        # Flash timer for alert
        self.flash_timer = QTimer()
        self.flash_timer.timeout.connect(self.toggle_flash)
        self.flash_state = False
        
        # Initial log message
        self.log_event("System initialized. Ready to start monitoring.")
        
    def start_monitoring(self):
        """Start video monitoring"""
        self.start_button.setEnabled(False)
        self.stop_button.setEnabled(True)
        
        # Create and start video processing thread
        self.video_thread = VideoProcessorThread(Config.VIDEO_SOURCE)
        self.video_thread.frame_processed.connect(self.update_display)
        self.video_thread.error_occurred.connect(self.handle_error)
        self.video_thread.start()
        
        self.log_event("Monitoring started.")
        self.status_label.setText("Status: Monitoring Active")
        self.status_label.setStyleSheet("color: green; padding: 10px;")
        
    def stop_monitoring(self):
        """Stop video monitoring"""
        if self.video_thread:
            self.video_thread.stop()
            self.video_thread.wait()
            self.video_thread = None
        
        self.start_button.setEnabled(True)
        self.stop_button.setEnabled(False)
        self.flash_timer.stop()
        
        self.log_event("Monitoring stopped.")
        self.status_label.setText("Status: Not Monitoring")
        self.status_label.setStyleSheet("color: gray; padding: 10px;")
        self.video_label.clear()
        self.video_label.setText("Video feed stopped")
        
    def update_display(self, results: dict):
        """Update GUI with detection results"""
        # Update video display
        frame = results['annotated_frame']
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        h, w, ch = rgb_frame.shape
        bytes_per_line = ch * w
        qt_image = QImage(rgb_frame.data, w, h, bytes_per_line, QImage.Format.Format_RGB888)
        pixmap = QPixmap.fromImage(qt_image)
        
        # Scale to fit label while maintaining aspect ratio
        scaled_pixmap = pixmap.scaled(
            self.video_label.size(),
            Qt.AspectRatioMode.KeepAspectRatio,
            Qt.TransformationMode.SmoothTransformation
        )
        self.video_label.setPixmap(scaled_pixmap)
        
        # Update status
        alert_info = results['alert']
        self.status_label.setText(alert_info['message'])
        
        # Set color and flash for critical alerts
        color = alert_info['color']
        if alert_info['tier'] >= 2:
            if not self.flash_timer.isActive():
                self.flash_timer.start(500)  # Flash every 500ms
        else:
            self.flash_timer.stop()
            self.status_label.setStyleSheet(f"color: {color}; padding: 10px; font-weight: bold; font-size: 14px;")
        
        # Update metrics
        metrics_text = f"""
        Person Detected: {'✓' if results['person_detected'] else '✗'}
        State: {results['state']}
        Eye Aspect Ratio: {results['ear']:.3f}
        Head Pitch: {results['pitch']:.1f}°
        Head Yaw: {results['yaw']:.1f}°
        Alert Tier: {alert_info['tier']}
        """
        self.metrics_label.setText(metrics_text)
        
        # Log significant events
        if alert_info['tier'] == 1 and not hasattr(self, '_last_tier1_log'):
            self.log_event(f"⚠️ TIER 1 WARNING: {results['state']} detected")
            self._last_tier1_log = True
        elif alert_info['tier'] == 2 and not hasattr(self, '_last_tier2_log'):
            self.log_event(f"🚨 TIER 2 ALARM: {results['state']} persisting!")
            self._last_tier2_log = True
            delattr(self, '_last_tier1_log')
        elif alert_info['tier'] == 3 and not hasattr(self, '_last_tier3_log'):
            self.log_event(f"🔴 TIER 3 CRITICAL: {results['state']} - Escalation required!")
            self._last_tier3_log = True
            delattr(self, '_last_tier2_log')
        elif alert_info['tier'] == 0:
            # Reset logging flags when back to normal
            if hasattr(self, '_last_tier1_log'):
                delattr(self, '_last_tier1_log')
            if hasattr(self, '_last_tier2_log'):
                delattr(self, '_last_tier2_log')
            if hasattr(self, '_last_tier3_log'):
                self.log_event(f"✓ Guard returned to attentive state")
                delattr(self, '_last_tier3_log')
    
    def toggle_flash(self):
        """Toggle flash effect for critical alerts"""
        self.flash_state = not self.flash_state
        if self.flash_state:
            self.status_label.setStyleSheet("color: white; background-color: red; padding: 10px; font-weight: bold; font-size: 14px;")
        else:
            self.status_label.setStyleSheet("color: red; padding: 10px; font-weight: bold; font-size: 14px;")
    
    def handle_error(self, error_message: str):
        """Handle errors from video processing thread"""
        self.log_event(f"❌ ERROR: {error_message}")
        self.stop_monitoring()
    
    def log_event(self, message: str):
        """Add an event to the log with timestamp"""
        timestamp = datetime.now().strftime("%H:%M:%S")
        log_entry = f"[{timestamp}] {message}"
        self.event_log.append(log_entry)
        # Auto-scroll to bottom
        self.event_log.verticalScrollBar().setValue(
            self.event_log.verticalScrollBar().maximum()
        )
    
    def closeEvent(self, event):
        """Clean up when closing the application"""
        if self.video_thread:
            self.video_thread.stop()
            self.video_thread.wait()
        event.accept()


# ==================== MAIN ENTRY POINT ====================
def main():
    """Main entry point for the application"""
    app = QApplication(sys.argv)
    
    # Set application style
    app.setStyle('Fusion')
    
    # Create and show main window
    window = SmartATMGuardian()
    window.show()
    
    sys.exit(app.exec())


if __name__ == "__main__":
    main()

In [ ]:
import tkinter as tk
from tkinter import messagebox
import cv2
import numpy as np
from PIL import Image, ImageTk
import threading
import time
import os

In [ ]:
import tkinter as tk
from tkinter import messagebox
import cv2
import numpy as np
from PIL import Image, ImageTk
import threading
import time
import os

# --- Configuration Constants ---

# How many pixels of movement is "not still"?
# This will need tuning based on your camera resolution and distance.
STILLNESS_THRESHOLD_PX = 10

# How many consecutive "still" frames trigger an alarm?
# (e.g., 100 frames at ~20fps is ~5 seconds)
SLEEP_ALARM_FRAMES = 100

class SleepingAlertApp:
    def __init__(self, window):
        """
        Initialize the application.
        """
        self.window = window
        self.window.title("Sleeping Alert System (Dev Phase)")
        self.window.geometry("800x700")

        # --- State Variables ---
        self.cap = None
        self.video_thread = None
        self.is_running = False

        # --- Face Detection ---
        # Use the built-in Haar Cascade data from OpenCV
        try:
            face_cascade_path = os.path.join(cv2.data.haarcascades, 'haarcascade_frontalface_default.xml')
            if not os.path.exists(face_cascade_path):
                raise IOError("Haar cascade file not found.")
            self.face_cascade = cv2.CascadeClassifier(face_cascade_path)
        except Exception as e:
            messagebox.showerror("Error", f"Failed to load Haar Cascade: {e}\nPlease ensure OpenCV is correctly installed.")
            self.window.destroy()
            return

        # --- "Sleeping" Logic State ---
        self.last_face_center = None
        self.stillness_counter = 0

        # --- GUI Elements ---
        self.main_frame = tk.Frame(self.window)
        self.main_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)

        # Video Display Label
        self.video_label = tk.Label(self.main_frame, bg="black")
        self.video_label.pack(fill=tk.BOTH, expand=True)

        # Status Label
        self.status_text = tk.StringVar()
        self.status_text.set("Ready. Press 'Start Camera' to begin.")
        self.status_label = tk.Label(self.main_frame, textvariable=self.status_text, font=("Arial", 14), pady=10)
        self.status_label.pack()

        # Control Buttons
        self.button_frame = tk.Frame(self.main_frame)
        self.button_frame.pack()

        self.start_button = tk.Button(self.button_frame, text="Start Camera", command=self.start_video_stream, font=("Arial", 12), width=15)
        self.start_button.pack(side=tk.LEFT, padx=5)

        self.stop_button = tk.Button(self.button_frame, text="Stop Camera", command=self.stop_video_stream, font=("Arial", 12), width=15, state=tk.DISABLED)
        self.stop_button.pack(side=tk.LEFT, padx=5)

        # Set up the close protocol
        self.window.protocol("WM_DELETE_WINDOW", self.on_closing)

    def start_video_stream(self):
        """
        Starts the video capture in a new thread.
        """
        if self.is_running:
            return

        try:
            self.cap = cv2.VideoCapture(0)  # Use 0 for laptop webcam
            if not self.cap.isOpened():
                raise IOError("Cannot open webcam.")
            
            self.is_running = True
            
            # Start the video processing thread
            # daemon=True ensures the thread will close when the main app closes
            self.video_thread = threading.Thread(target=self.video_loop, daemon=True)
            self.video_thread.start()
            
            self.start_button.config(state=tk.DISABLED)
            self.stop_button.config(state=tk.NORMAL)
            self.status_text.set("Camera running... looking for person.")
        
        except IOError as e:
            messagebox.showerror("Webcam Error", str(e))
            if self.cap:
                self.cap.release()

    def stop_video_stream(self):
        """
        Signals the video loop to stop.
        """
        self.is_running = False
        
        # The thread will see self.is_running is False and exit
        # We wait a moment for the thread to finish
        if self.video_thread:
            self.video_thread.join(timeout=0.5) 
            
        if self.cap:
            self.cap.release()

        self.start_button.config(state=tk.NORMAL)
        self.stop_button.config(state=tk.DISABLED)
        self.status_text.set("Camera stopped.")
        self.video_label.config(image=None) # Clear the image
        
        # Reset sleeping logic state
        self.last_face_center = None
        self.stillness_counter = 0

    def video_loop(self):
        """
        The main loop for video processing. Runs in a separate thread.
        """
        while self.is_running:
            try:
                ret, frame = self.cap.read()
                if not ret:
                    self.status_text.set("Error: Can't read from camera.")
                    time.sleep(0.5)
                    continue

                # Flip for a "mirror" view, which is more intuitive
                frame = cv2.flip(frame, 1)
                
                # Process the frame
                processed_frame, status = self.detect_sleeping(frame)

                # Update the status text
                self.status_text.set(status)

                # Convert the OpenCV (BGR) frame to a PIL (RGB) image
                cv_img = cv2.cvtColor(processed_frame, cv2.COLOR_BGR2RGB)
                pil_img = Image.fromarray(cv_img)
                
                # Resize image to fit the label (optional, but good for layout)
                w, h = self.video_label.winfo_width(), self.video_label.winfo_height()
                if w > 1 and h > 1: # Avoid division by zero on init
                     pil_img = pil_img.resize((w, h), Image.Resampling.LANCZOS)

                # Convert PIL image to Tkinter-compatible image
                imgtk = ImageTk.PhotoImage(image=pil_img)

                # Update the video label in the GUI
                # This must be done from the main thread, but tkinter seems to handle this call
                self.video_label.imgtk = imgtk
                self.video_label.configure(image=imgtk)

            except Exception as e:
                print(f"Error in video loop: {e}")
                self.is_running = False
            
            # Control loop speed slightly
            time.sleep(0.01) # ~100fps theoretical max, but processing will slow it down

        print("Video loop stopped.")

    def detect_sleeping(self, frame):
        """
        Performs face detection and sleeping logic on a single frame.
        """
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = self.face_cascade.detectMultiScale(
            gray,
            scaleFactor=1.1,
            minNeighbors=5,
            minSize=(50, 50) # Don't detect tiny faces
        )

        current_status = "No Person Detected"
        alert_triggered = False

        if len(faces) > 0:
            # We only track the largest/first face
            (x, y, w, h) = faces[0]
            
            # Calculate the center of the face
            center_x = x + w // 2
            center_y = y + h // 2
            current_center = (center_x, center_y)

            current_status = "Monitoring: Person Active"

            # Compare with the last known center
            if self.last_face_center is not None:
                # Calculate Euclidean distance
                dist = np.linalg.norm(np.array(current_center) - np.array(self.last_face_center))

                if dist < STILLNESS_THRESHOLD_PX:
                    # Person is still
                    self.stillness_counter += 1
                else:
                    # Person moved, reset counter
                    self.stillness_counter = 0
            
            # Update the last known center
            self.last_face_center = current_center

            # Check if the stillness has crossed the alarm threshold
            if self.stillness_counter > SLEEP_ALARM_FRAMES:
                current_status = "!!! ALERT: SLEEPING DETECTED !!!"
                alert_triggered = True
            
            # --- Draw on the frame ---
            if alert_triggered:
                # Draw a bright red box for alert
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 3)
                cv2.putText(frame, "ALERT!", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
                # We could also play a sound here
                # e.g., import winsound; winsound.Beep(1000, 500)
            else:
                # Draw a standard blue box
                cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
                cv2.putText(frame, f"Stillness: {self.stillness_counter}/{SLEEP_ALARM_FRAMES}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)

        else:
            # No face detected, reset the logic
            self.last_face_center = None
            self.stillness_counter = 0
            current_status = "No Person Detected"

        return frame, current_status

    def on_closing(self):
        """
        Handles the window close event.
        """
        if messagebox.askokcancel("Quit", "Do you want to exit the application?"):
            self.stop_video_stream()
            self.window.destroy()

# --- Main execution ---
if __name__ == "__main__":
    try:
        root = tk.Tk()
        app = SleepingAlertApp(root)
        root.mainloop()
    except Exception as e:
        print(f"An error occurred: {e}")

In [1]:
import tkinter as tk
from tkinter import messagebox, simpledialog  # Import simpledialog
import cv2
import numpy as np
from PIL import Image, ImageTk
import threading
import time
import os

# --- Configuration Constants ---

# How many pixels of movement is "not still"?
# This will need tuning based on your camera resolution and distance.
STILLNESS_THRESHOLD_PX = 10

# How many consecutive "still" frames trigger an alarm?
# (e.g., 100 frames at ~20fps is ~5 seconds)
SLEEP_ALARM_FRAMES = 100

# New constant for tracking
# If the closest-found face is further than this from the last frame,
# we assume it's a new person or the tracker lost the original.
MAX_TRACKING_JUMP_PX = 150

class SleepingAlertApp:
    def __init__(self, window):
        """
        Initialize the application.
        """
        self.window = window
        self.window.title("Sleeping Alert System (Dev Phase)")
        self.window.geometry("800x700")

        # --- State Variables ---
        self.cap = None
        self.video_thread = None
        self.is_running = False

        # --- Face Detection ---
        # Use the built-in Haar Cascade data from OpenCV
        try:
            face_cascade_path = os.path.join(cv2.data.haarcascades, 'haarcascade_frontalface_default.xml')
            if not os.path.exists(face_cascade_path):
                raise IOError("Haar cascade file not found.")
            self.face_cascade = cv2.CascadeClassifier(face_cascade_path)
        except Exception as e:
            messagebox.showerror("Error", f"Failed to load Haar Cascade: {e}\nPlease ensure OpenCV is correctly installed.")
            self.window.destroy()
            return

        # --- "Sleeping" Logic State ---
        self.last_face_center = None
        self.stillness_counter = 0
        
        # --- New Registration State ---
        self.is_monitoring = False  # Are we actively monitoring a registered person?
        self.registered_face_center_guess = None # The last known center of the monitored person
        self.registered_person_name = None # Store the registered person's name
        self.current_frame_for_registration = None # Temp stores frame for registration
        self.registration_lock = threading.Lock() # Lock for accessing the frame

        # --- GUI Elements ---
        self.main_frame = tk.Frame(self.window)
        self.main_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)

        # Video Display Label
        self.video_label = tk.Label(self.main_frame, bg="black")
        self.video_label.pack(fill=tk.BOTH, expand=True)

        # Status Label
        self.status_text = tk.StringVar()
        self.status_text.set("Ready. Press 'Start Camera' to begin.")
        self.status_label = tk.Label(self.main_frame, textvariable=self.status_text, font=("Arial", 14), pady=10)
        self.status_label.pack()

        # Control Buttons
        self.button_frame = tk.Frame(self.main_frame)
        self.button_frame.pack()

        self.start_button = tk.Button(self.button_frame, text="Start Camera", command=self.start_video_stream, font=("Arial", 12), width=15)
        self.start_button.pack(side=tk.LEFT, padx=5)

        self.stop_button = tk.Button(self.button_frame, text="Stop Camera", command=self.stop_video_stream, font=("Arial", 12), width=15, state=tk.DISABLED)
        self.stop_button.pack(side=tk.LEFT, padx=5)
        
        # --- New Registration Buttons ---
        self.register_button = tk.Button(self.button_frame, text="Register & Monitor", command=self.register_person, font=("Arial", 12), width=18, state=tk.DISABLED)
        self.register_button.pack(side=tk.LEFT, padx=5)

        self.clear_button = tk.Button(self.button_frame, text="Clear Registration", command=self.clear_registration, font=("Arial", 12), width=18, state=tk.DISABLED)
        self.clear_button.pack(side=tk.LEFT, padx=5)


        # Set up the close protocol
        self.window.protocol("WM_DELETE_WINDOW", self.on_closing)

    def start_video_stream(self):
        """
        Starts the video capture in a new thread.
        """
        if self.is_running:
            return

        try:
            self.cap = cv2.VideoCapture(0)  # Use 0 for laptop webcam
            if not self.cap.isOpened():
                raise IOError("Cannot open webcam.")
            
            self.is_running = True
            
            # Start the video processing thread
            # daemon=True ensures the thread will close when the main app closes
            self.video_thread = threading.Thread(target=self.video_loop, daemon=True)
            self.video_thread.start()
            
            self.start_button.config(state=tk.DISABLED)
            self.stop_button.config(state=tk.NORMAL)
            self.register_button.config(state=tk.NORMAL) # Enable registration
            self.status_text.set("Camera running. Click 'Register & Monitor'.")
        
        except IOError as e:
            messagebox.showerror("Webcam Error", str(e))
            if self.cap:
                self.cap.release()

    def stop_video_stream(self):
        """
        Signals the video loop to stop.
        """
        self.is_running = False
        
        # The thread will see self.is_running is False and exit
        # We wait a moment for the thread to finish
        if self.video_thread:
            self.video_thread.join(timeout=0.5) 
            
        if self.cap:
            self.cap.release()

        self.start_button.config(state=tk.NORMAL)
        self.stop_button.config(state=tk.DISABLED)
        self.status_text.set("Camera stopped.")
        self.video_label.config(image=None) # Clear the image
        
        # --- Reset all states ---
        self.is_monitoring = False
        self.registered_face_center_guess = None
        self.registered_person_name = None # Clear name
        self.last_face_center = None
        self.stillness_counter = 0
        self.register_button.config(state=tk.DISABLED)
        self.clear_button.config(state=tk.DISABLED)

    def video_loop(self):
        """
        The main loop for video processing. Runs in a separate thread.
        """
        while self.is_running:
            try:
                ret, frame = self.cap.read()
                if not ret:
                    self.status_text.set("Error: Can't read from camera.")
                    time.sleep(0.5)
                    continue

                # Flip for a "mirror" view, which is more intuitive
                frame = cv2.flip(frame, 1)
                
                # Store a copy of the frame for the registration function
                with self.registration_lock:
                    self.current_frame_for_registration = frame.copy()
                
                # Process the frame
                processed_frame, status = self.process_frame_logic(frame)

                # Update the status text
                self.status_text.set(status)

                # Convert the OpenCV (BGR) frame to a PIL (RGB) image
                cv_img = cv2.cvtColor(processed_frame, cv2.COLOR_BGR2RGB)
                pil_img = Image.fromarray(cv_img)
                
                # Resize image to fit the label (optional, but good for layout)
                w, h = self.video_label.winfo_width(), self.video_label.winfo_height()
                if w > 1 and h > 1: # Avoid division by zero on init
                     pil_img = pil_img.resize((w, h), Image.Resampling.LANCZOS)

                # Convert PIL image to Tkinter-compatible image
                imgtk = ImageTk.PhotoImage(image=pil_img)

                # Update the video label in the GUI
                # This must be done from the main thread, but tkinter seems to handle this call
                self.video_label.imgtk = imgtk
                self.video_label.configure(image=imgtk)

            except Exception as e:
                print(f"Error in video loop: {e}")
                self.is_running = False
            
            # Control loop speed slightly
            time.sleep(0.01) # ~100fps theoretical max, but processing will slow it down

        print("Video loop stopped.")

    def register_person(self):
        """
        Registers the largest face currently in the frame for monitoring.
        """
        with self.registration_lock:
            if self.current_frame_for_registration is None:
                messagebox.showwarning("Registration Error", "Camera not ready. Please try again.")
                return
            
            # Use the stored frame to find a face
            gray = cv2.cvtColor(self.current_frame_for_registration, cv2.COLOR_BGR2GRAY)
            faces = self.face_cascade.detectMultiScale(
                gray,
                scaleFactor=1.1,
                minNeighbors=5,
                minSize=(50, 50)
            )

        if len(faces) == 0:
            messagebox.showwarning("Registration Error", "No person detected in the frame. Please face the camera and try again.")
            return

        # Register the largest face
        # We sort by area to be sure we get the main person
        faces_by_area = sorted(faces, key=lambda f: f[2] * f[3], reverse=True)
        (x, y, w, h) = faces_by_area[0]

        center_x = x + w // 2
        center_y = y + h // 2
        
        # --- NEW: Ask for the person's name ---
        name = simpledialog.askstring("Register Person", "Enter the person's name:", parent=self.window)
        
        if not name:
            messagebox.showwarning("Registration Cancelled", "Registration was cancelled (no name provided).")
            return
        
        # --- Lock in the registration ---
        self.is_monitoring = True
        self.registered_person_name = name
        self.registered_face_center_guess = (center_x, center_y)
        self.last_face_center = (center_x, center_y) # Start stillness check from this point
        self.stillness_counter = 0

        # Update GUI
        self.register_button.config(state=tk.DISABLED)
        self.clear_button.config(state=tk.NORMAL)
        self.status_text.set(f"Person registered: {self.registered_person_name}. Actively monitoring.")
        messagebox.showinfo("Registration Complete", f"{self.registered_person_name} registered. Monitoring has started.")

    def clear_registration(self):
        """
        Clears the current registration and stops monitoring.
        """
        self.is_monitoring = False
        self.registered_face_center_guess = None
        self.registered_person_name = None # Clear name
        self.last_face_center = None
        self.stillness_counter = 0

        # Update GUI
        if self.is_running: # Only enable if camera is on
            self.register_button.config(state=tk.NORMAL)
        self.clear_button.config(state=tk.DISABLED)
        self.status_text.set("Monitoring stopped. Ready to register a new person.")


    def process_frame_logic(self, frame):
        """
        Detects faces and applies monitoring logic based on registration status.
        This function replaces the old 'detect_sleeping'.
        """
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = self.face_cascade.detectMultiScale(
            gray,
            scaleFactor=1.1,
            minNeighbors=5,
            minSize=(50, 50) # Don't detect tiny faces
        )

        # If not monitoring, just show all faces found
        if not self.is_monitoring:
            current_status = "Ready to register."
            if self.is_running: # Check if camera is on
                current_status = "Click 'Register & Monitor' to begin."
            
            for (x, y, w, h) in faces:
                cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 192, 0), 2) # Light blue box
            
            return frame, current_status

        # --- If we ARE monitoring ---
        
        if len(faces) == 0:
            # We were monitoring, but now the person is gone
            current_status = f"MONITORING: {self.registered_person_name} lost!"
            self.last_face_center = None # Stop stillness counter
            self.stillness_counter = 0
            return frame, current_status

        # Find the face closest to our last known center
        min_dist = float('inf')
        tracked_face = None
        current_center = None

        for (x, y, w, h) in faces:
            center = (x + w // 2, y + h // 2)
            dist = np.linalg.norm(np.array(center) - np.array(self.registered_face_center_guess))
            
            if dist < min_dist:
                min_dist = dist
                tracked_face = (x, y, w, h)
                current_center = center
        
        # Check if the closest face is "reasonably" close.
        # If not, it's probably a different person.
        if min_dist > MAX_TRACKING_JUMP_PX:
             # The person we found is too far from the last spot.
             current_status = f"MONITORING: {self.registered_person_name} lost! (New face detected)"
             self.last_face_center = None
             self.stillness_counter = 0
             # Draw a box on the *new* face
             (x, y, w, h) = tracked_face
             cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 165, 255), 2) # Orange box
             return frame, current_status

        # --- We have successfully re-acquired the tracked face ---
        (x, y, w, h) = tracked_face
        
        # Update our guess for the next frame
        self.registered_face_center_guess = current_center
        
        current_status = f"Monitoring: {self.registered_person_name} (Active)"
        alert_triggered = False

        # Compare with the last known center
        if self.last_face_center is not None:
            dist_moved = np.linalg.norm(np.array(current_center) - np.array(self.last_face_center))

            if dist_moved < STILLNESS_THRESHOLD_PX:
                # Person is still
                self.stillness_counter += 1
            else:
                # Person moved, reset counter
                self.stillness_counter = 0
        
        # Update the last known center for the *next* frame's comparison
        self.last_face_center = current_center

        # Check if the stillness has crossed the alarm threshold
        if self.stillness_counter > SLEEP_ALARM_FRAMES:
            current_status = f"!!! ALERT: {self.registered_person_name} IS SLEEPING !!!"
            alert_triggered = True
        
        # --- Draw on the frame ---
        if alert_triggered:
            # Draw a bright red box for alert
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 3)
            cv2.putText(frame, f"ALERT: {self.registered_person_name}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
        else:
            # Draw a standard blue box for the tracked person
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
            cv2.putText(frame, self.registered_person_name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)
            cv2.putText(frame, f"Stillness: {self.stillness_counter}/{SLEEP_ALARM_FRAMES}", (x, y + h + 25), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)

        return frame, current_status


    def detect_sleeping(self, frame):
        """
        This function is no longer called directly by the video_loop.
        Its logic has been moved into process_frame_logic.
        """
        pass # Kept to show what was replaced
        

    def on_closing(self):
        """
        Handles the window close event.
        """
        if messagebox.askokcancel("Quit", "Do you want to exit the application?"):
            self.stop_video_stream()
            self.window.destroy()

# --- Main execution ---
if __name__ == "__main__":
    try:
        root = tk.Tk()
        app = SleepingAlertApp(root)
        root.mainloop()
    except Exception as e:
        print(f"An error occurred: {e}")